<a href="https://colab.research.google.com/github/VLADpit/MyGoogleColab/blob/main/rhytms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Импорты и подгрузка необходимых библиотек

In [1]:
!pip install pyedflib numpy scipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 31.1 MB/s eta 0:00:00


In [2]:
import numpy as np
from scipy.signal import butter, filtfilt, welch
import pyedflib

# Функции

Полосовой фильтр на основе [фильтра Баттерфорта](https://ru.wikipedia.org/wiki/Фильтр_Баттерворта?ysclid=mldttz69kp980612220) для обрезвния частот до 1-40Гц(физиологические ритмы)


In [ ]:
def bandfilter(signal, fs):
    nyq = 0.5 * fs
    low, high = 1 / nyq, 40 / nyq
    b, a = butter(4, [low, high], btype='band')
    return filtfilt(b, a, signal)